# Agents

Agents are composed of body parts and sensors.
We will use an already implemented agent composed of a Base and a Head.

We are going to attach sensors to this agent.

In [1]:
from simple_playgrounds.playgrounds import SingleRoom
from simple_playgrounds import Engine
from simple_playgrounds.controllers import Keyboard
from simple_playgrounds.entities.agents import BaseAgent, HeadAgent

# to display the sensors
import cv2


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Linux (64bit) [/home/michael/.local/lib/python3.6/site-packages/pymunk/libchipmunk.so]


In [2]:
my_playground = SingleRoom(size=(200, 200))
my_agent = HeadAgent(controller=Keyboard())

# we use the option screen=True to use a keyboard controlled agent later on.
engine = Engine(time_limit=1000000, agents=my_agent, playground= my_playground, screen=True)

engine.display_full_scene()

## Adding scene elements 

Now lets add some scene element to the playground, that the agent will be able to perceive.

In [3]:
from simple_playgrounds.entities.scene_elements import Basic

circular_object = Basic((30, 30, 0.5),default_config_key='pentagon', graspable=True, mass = 10)
my_playground.add_scene_element(circular_object)
engine.display_full_scene()

circular_object = Basic((30, 160, -0.5), physical_shape='circle', radius=10, texture = [120, 0, 230])
my_playground.add_scene_element(circular_object)
engine.display_full_scene()


## Adding sensors to the agent 

Ok Great! Now we will add 2 rgb sensors: one attached to the head, one attached to the body.

Note that the properties of the sensors can be changed with keyword arguments.

Move the agent with keyboard arrows, and its head with n and m

In [4]:

from simple_playgrounds.entities.agents.sensors import RgbSensor
my_agent.add_sensor(RgbSensor(my_agent.head, invisible_elements=my_agent.parts, fov=180))
my_agent.add_sensor(RgbSensor(my_agent.base_platform, invisible_elements=my_agent.parts, fov=270))

while engine.game_on:
    
    engine.display_full_scene()

    actions = {}
    for agent in engine.agents:
        actions[agent.name] = agent.controller.generate_actions()

    engine.step(actions)
    engine.update_observations()
    
    cv2.imshow('sensor', engine.generate_sensor_image(my_agent))
    cv2.waitKey(20)
    
engine.terminate()
cv2.destroyAllWindows()

## Lets add all sensors and move around

We load a test playground.

In [5]:
from simple_playgrounds.playgrounds.collection.test import Basics
from simple_playgrounds.entities.agents.sensors import RgbSensor, TopdownSensor, TouchSensor, LidarRays, DepthSensor, DistanceArraySensor

my_playground = Basics()
my_agent = BaseAgent(controller=Keyboard())

# we use the option screen=True to use a keyboard controlled agent later on.
engine = Engine(time_limit=1000000, agents=my_agent, playground= my_playground, screen=True)

my_agent.add_sensor(RgbSensor(my_agent.base_platform))
my_agent.add_sensor(DepthSensor(my_agent.base_platform))
my_agent.add_sensor(TouchSensor(my_agent.base_platform, range=3))
my_agent.add_sensor(DistanceArraySensor(my_agent.base_platform))
my_agent.add_sensor(LidarRays(my_agent.base_platform, allow_duplicates=True))
my_agent.add_sensor(TopdownSensor(my_agent.base_platform))

while engine.game_on:
    
    engine.display_full_scene()

    actions = {}
    for agent in engine.agents:
        actions[agent.name] = agent.controller.generate_actions()

    engine.step(actions)
    engine.update_observations()
    
    cv2.imshow('sensor', engine.generate_sensor_image(my_agent))
    cv2.waitKey(20)
    
engine.terminate()
cv2.destroyAllWindows()


## Funny addition

Sensors are quite generic, so we could in principle add one to a scene element.
It would act as a camera, that the agent has access to.

Here, we add a sensor to an object which is graspable.

In [6]:
from simple_playgrounds.entities.agents import BaseInteractiveAgent

my_playground = Basics()
my_agent = BaseInteractiveAgent(controller=Keyboard())

# we use the option screen=True to use a keyboard controlled agent later on.
engine = Engine(time_limit=1000000, agents=my_agent, playground= my_playground, screen=True)

circular_object = Basic((30, 30, 0.5), physical_shape='circle', radius=10, texture = [120, 230, 0], graspable = True, mass=5)
my_playground.add_scene_element(circular_object)

camera = RgbSensor(circular_object)

my_agent.add_sensor(camera)

while engine.game_on:
    
    engine.display_full_scene()

    actions = {}
    for agent in engine.agents:
        actions[agent.name] = agent.controller.generate_actions()

    engine.step(actions)
    engine.update_observations()
    
    cv2.imshow('sensor', engine.generate_sensor_image(my_agent))
    cv2.waitKey(20)
    
engine.terminate()
cv2.destroyAllWindows()
